In [1]:
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))

In [2]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import sys, os
import copy

In [3]:
%load_ext qat.core.magic
%matplotlib inline

The qat.core.magic extension is already loaded. To reload it, use:
  %reload_ext qat.core.magic


In [4]:
#QPU connection
QLMASS = False
if QLMASS:
    try:
        from qat.qlmaas import QLMaaSConnection
        connection = QLMaaSConnection()
        LinAlg = connection.get_qpu("qat.qpus:LinAlg")
        lineal_qpu = LinAlg()
    except (ImportError, OSError) as e:
        print('Problem: usin PyLinalg')
        from qat.qpus import PyLinalg
        lineal_qpu = PyLinalg()
else:
    print('User Forces: PyLinalg')
    from qat.qpus import PyLinalg
    lineal_qpu = PyLinalg() 

User Forces: PyLinalg


In [5]:
sys.path.append('/home/cesga/gferro/NEASQC/PhaseAmplification/')

In [7]:
from QuantumMultiplexors_Module import expectation_loading_data
from PhaseAmplification_Module import load_q_gate
from AuxiliarFunctions import  get_histogram, postprocess_results, test_bins, run_job
def p(x):
    return x*x
def f(x):
    return np.sin(x)

In [8]:
def Launch(n_qbits = 6):
    #The number of bins 
    m_bins = 2**n_qbits
    LowerLimit = 0.0
    UpperLimit = 1.0 
    #Discretizing Data
    X, p_X = get_histogram(p, LowerLimit, UpperLimit, m_bins)
    f_X = f(X)
    #Create Qunatum Program and Quantum Gates
    Qprog, P_Gate, R_gate = expectation_loading_data(p_X, f_X)
    Q_Gate = load_q_gate(P_Gate, R_gate)
    return Qprog, Q_Gate
    
def get_staff(InputPdf):
    pdf = InputPdf.copy(deep=True)
    #Get Unitary theta
    pdf['UnitaryTheta'] = 2*np.pi*pdf['Phi']
    pdf['QAA_Theta'] = np.pi*pdf['Phi']
    pdf['Theta'] = np.pi*pdf['Phi']
    pdf['Theta'].where(pdf['Theta']< 0.5*np.pi, np.pi-pdf['Theta'], inplace=True)
    
    pdf['E_p(f)'] = np.sin(pdf['Theta'])**2
    return pdf

In [9]:
n_qbits = 6
n_cbits = 6
q_prog, q_gate = Launch(n_qbits)

TypeError: cannot unpack non-iterable ParamGate object

In [ ]:
c = q_prog.to_circ()
%qatdisplay c
%qatdisplay q_gate

In [ ]:
circuit = q_prog.to_circ(submatrices_only=True)

In [ ]:
job = circuit.to_job()

In [ ]:
result = run_job(lineal_qpu.submit(job))

In [ ]:
initial_state = postprocess_results(result)

In [ ]:
initial_state

In [ ]:
from NoMPI_IQAE import step_iqae_easy

In [ ]:
class_bits = [0 for i in range(n_cbits)]
l=0
step_job, step_circuit = step_iqae_easy(q_prog, q_gate, class_bits, l)

In [ ]:
%qatdisplay step_circuit

In [ ]:
job = step_circuit.to_job()

In [ ]:
result = run_job(lineal_qpu.submit(job))

In [ ]:
first_state = postprocess_results(result)

In [ ]:
first_state

In [ ]:
result[0]

In [ ]:
step_job, step_circuit = step_iqae_easy(q_prog, q_gate, class_bits, l=0)
result = run_job(lineal_qpu.submit(step_job))

In [ ]:
class_bits[0] = result[0].state.int

In [ ]:
step_job, step_circuit = step_iqae_easy(q_prog, q_gate, class_bits, l=1)
result = run_job(lineal_qpu.submit(step_job))

In [ ]:
%qatdisplay step_circuit

In [ ]:
class_bits = [0 for i in range(n_cbits)]
#c_bits = q_prog.calloc(n_cbits)
list_of_circuits = []
for l in range(n_cbits):
    step_job, step_circuit = step_iqae_easy(q_prog, q_gate, class_bits, l)
    list_of_circuits.append(step_circuit)
    result = run_job(lineal_qpu.submit(step_job))
    meas_cb = result[0].state.int
    class_bits[l] = meas_cb

In [ ]:
%qatdisplay circuit

In [ ]:
from NoMPI_IQAE import Do

## 1. Una Iteración

In [ ]:
n_qbits = 6
n_cbits = 6
Qprog, Q_Gate = Launch(n_qbits)
pdf, _, ListOfCircuits = Do(Qprog, Q_Gate, n_cbits)
pdf = get_staff(pdf)

In [ ]:
pdf

In [ ]:
c = ListOfCircuits[0]
%qatdisplay c

## 2. Varias  Iteraciones

In [ ]:
n_qbits = 6
n_cbits = 6


list_of_pdfs = []
lc = []
for i in range(50):
    print(i)
    Qprog, Q_Gate = Launch(n_qbits)
    pdf, _, sc = Do(Qprog, Q_Gate, n_cbits)
    list_of_pdfs.append(get_staff(pdf))
    lc.append(sc)
pdf = pd.concat(list_of_pdfs)
pdf.reset_index(drop=True, inplace=True)

In [ ]:
c = lc[-1][5]
%qatdisplay c

In [ ]:
plt.hist(pdf['E_p(f)'], bins = 50)

In [ ]:
plt.hist(pdf['Theta'], bins = 50)

## Qiskit

In [ ]:
from qat.lang.AQASM import Program, H, X, PH
nq = 2
nc = 6
q_prog = Program()
q_bits = q_prog.qalloc(nq)
for i in range(nq):
    q_prog.apply(X, q_bits[i])
q_gate = PH(np.pi/4.0).ctrl()

In [ ]:
c = q_prog.to_circ()
%qatdisplay c

In [ ]:
pdf, _, ListOfCircuits = Do(q_prog, q_gate, nc)

In [ ]:
pdf

In [ ]:
c = ListOfCircuits[0]
%qatdisplay c

In [ ]:
list_of_pdfs = []
for i in range(50):
    print(i)
    pdf, _, ListOfCircuits = Do(q_prog, q_gate, nc)
    list_of_pdfs.append(pdf)
pdf = pd.concat(list_of_pdfs)
pdf.reset_index(drop=True, inplace=True)       

In [ ]:
ci = ListOfCircuits[0]

%qatdisplay ci

In [ ]:
pdf['Phi'].describe()